# Using GeoJSON Point Features with Markers

In [3]:
import folium
import geopandas as gpd

In [4]:
gdf = gpd.read_file("examples/data/subwaystations.geojson")

In [26]:
gdf.head()

,name,url,line,objectid,notes,geometry,href,service_level,lines_served
0,Astor Pl,http://web.mta.info/nyct/service/,4-6-6 Express,1,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-73.99107 40.73005),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,3
1,Canal St,http://web.mta.info/nyct/service/,4-6-6 Express,2,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-74.00019 40.71880),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,3
2,50th St,http://web.mta.info/nyct/service/,1-2,3,"1-all times, 2-nights",POINT (-73.98385 40.76173),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,2
3,Bergen St,http://web.mta.info/nyct/service/,2-3-4,4,"4-nights, 3-all other times, 2-all times",POINT (-73.97500 40.68086),"<a href=""http://web.mta.info/nyct/service/"">ht...",2,3
4,Pennsylvania Ave,http://web.mta.info/nyct/service/,3-4,5,"4-nights, 3-all other times",POINT (-73.89489 40.66471),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,2


In [6]:
gdf['href']='<a href="'+gdf.url+'">'+ gdf.url +"</a>"
gdf['service_level'] = gdf.notes.str.split(', ').apply(lambda x: len([v for v in x if "all" in v]))
gdf['lines_served'] = gdf.line.str.split('-').apply(lambda x: len(x))

,name,url,line,objectid,notes,geometry,href,service_level,lines_served
0,Astor Pl,http://web.mta.info/nyct/service/,4-6-6 Express,1,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-73.99107 40.73005),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,3
1,Canal St,http://web.mta.info/nyct/service/,4-6-6 Express,2,"4 nights, 6-all times, 6 Express-weekdays AM s...",POINT (-74.00019 40.71880),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,3
2,50th St,http://web.mta.info/nyct/service/,1-2,3,"1-all times, 2-nights",POINT (-73.98385 40.76173),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,2
3,Bergen St,http://web.mta.info/nyct/service/,2-3-4,4,"4-nights, 3-all other times, 2-all times",POINT (-73.97500 40.68086),"<a href=""http://web.mta.info/nyct/service/"">ht...",2,3
4,Pennsylvania Ave,http://web.mta.info/nyct/service/,3-4,5,"4-nights, 3-all other times",POINT (-73.89489 40.66471),"<a href=""http://web.mta.info/nyct/service/"">ht...",1,2


In [27]:
service_levels = gdf.service_level.unique().tolist()
service_levels

[1, 2, 3, 0]

In [28]:

colors = ["orange", "yellow", "green", "blue"]

# Use a Circle as a Marker

In [29]:
m = folium.Map(location=[40.75,-73.95], zoom_start=13)

stations = folium.GeoJson(
    gdf,
    name="Subway Stations",
    marker = folium.Circle(radius=4, fill_color="orange", fill_opacity=0.4, color="black", weight=1),
    tooltip=folium.GeoJsonTooltip(fields=["name", "line", "notes"]),
    zoom_on_click=True,
    popup=folium.GeoJsonPopup(fields=["name", "line", "href", "notes"]),
    style_function=lambda x: {"markerColor": "yellow", "fillColor": colors[x['properties']['service_level']], "radius": (x['properties']['lines_served'])*30},
    highlight_function=lambda x: {"fillOpacity": 0.8}
).add_to(m)

m


## Or use a DivIcon

In [30]:
m = folium.Map(location=[40.75,-73.95], zoom_start=13)

stations = folium.GeoJson(
    gdf,
    name="Subway Stations",
    # marker=folium.Marker(icon=folium.DivIcon(icon_size=(30,30))),
    marker = folium.Marker(icon=folium.DivIcon()),
    tooltip=folium.GeoJsonTooltip(fields=["name", "line", "notes"]),
    zoom_on_click=True,
    popup=folium.GeoJsonPopup(fields=["name", "line", "href", "notes"]),
    style_function=lambda x: {'html': f"""<a href="{x['properties']['url']}"><div style="font-size: 0.8em;"><div style="width: 10px; height: 10px; border: 1px solid black; border-radius: 5px; background-color: orange;"></div>{x['properties']['name']}</div></a>"""}
).add_to(m)

m